In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras.optimizers import SGD, Adam
import numpy as np
from deep_tools.training_callbacks import *
import math
from keras.utils import to_categorical
import random
# fix random seed for reproducibility
np.random.seed(7)


In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.

      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)

      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.

    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)

    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [ ]:
from keras import backend as K
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)

In [ ]:
# create model
model = Sequential()
model.add(Dense(2048, input_dim=4096, activation='relu'))
model.add(Dropout(0.2))

# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# # create model
# model = Sequential()
# model.add(Dense(2048, input_dim=4096, activation='relu'))
# model.add(Dropout(0.2))

# # model.add(Dense(2048, activation='relu'))
# # model.add(Dropout(0.2))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.2))
# # model.add(Dense(512, activation='relu'))
# # model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# # model.add(Dropout(0.2))
# model.add(Dense(5, activation='softmax'))

In [ ]:
# model.summary()

In [ ]:
# Compile model
learning_rate = 0.01
# decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=0, nesterov=False)
# model.compile(loss=binary_focal_loss(), optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
def step_decay(epoch):
   initial_lrate = 0.10
   drop = 0.8
   epochs_drop = 10.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate

def exp_decay(epoch):
   initial_lrate = 0.01
   k = 0.1
   lrate = initial_lrate * math.exp(-k*epoch)
   return lrate

# lrate = LearningRateScheduler(exp_decay)
lrate = LearningRateScheduler(step_decay)

# class LossHistory(Callback):
#     def on_train_begin(self, logs={}):
#        self.losses = []
#        self.lr = []
 
#     def on_epoch_end(self, batch, logs={}):
#        self.losses.append(logs.get("val_loss"))
#        self.lr.append(step_decay(len(self.losses)))

In [ ]:
acc_callback = TrainingAccuracyPlotter()
loss_plotter = TrainingLossPlotter()
time_plotter = TimeLogCallback()


checkpoint = checkpoint_classification_callback(
    checkpoint_path="/home/wc-gpu/MasterThesis/models/research/object_detection/box_classification_checkpoint", 
    monitor='val_acc',
    save_best_only=True)
callbacks = [time_plotter, acc_callback, checkpoint]

In [ ]:
data = np.load(
    "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000/old_ds/dataset_prod_new_model_binary.pkl")

In [ ]:
data = np.load(
    "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000/dataset_prod_new_model_binary_2prev_session_split.pkl")

In [ ]:
Xdict = data[0]
Ydict = data[1]
sessions = Xdict.keys()

In [ ]:
# random.sample(sessions, 50)

In [ ]:
val = set()

In [ ]:
xtrain = []
xval = []
ytrain = []
yval = []


for s in sessions:
    if s in val:
        xval.extend(Xdict[s])
        yval.extend(Ydict[s])
    else:
        xtrain.extend(Xdict[s])
        ytrain.extend(Ydict[s])

xtrain = np.asarray(xtrain)
xval = np.asarray(xval)
ytrain = np.asarray(ytrain)
yval = np.asarray(yval)
    

In [ ]:
x = xtrain
y = ytrain

In [ ]:
x, y = data

In [ ]:
x.shape

In [ ]:
len(np.where(y == 0)[0])

In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y)
                                               ,y)

In [ ]:
class_weight = [ 0.001,  1]

In [ ]:
# class_weight = [ 0.01,  0.9,  0.9, 100, 100]

In [ ]:
len(np.where(y==1)[0])

In [ ]:
def normalize(x):
    mn = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    x = (x - mn)/std
    return mn, std

In [ ]:
# normalize(xval)
normalize(x)

In [ ]:
negatives = np.where(y == 0)[0]
negatives = negatives[np.random.choice(negatives.shape[0], len(np.where(y==1)[0]), replace=False)]
positives = np.where(y != 0)[0]
idxs = np.concatenate([negatives, positives])
np.random.shuffle(idxs)
# x[np.where(x == 0)[0]] = .1
x = x[idxs]
y = y[idxs]

In [ ]:
y_cat = to_categorical(y, num_classes=None, dtype='float32')


In [ ]:
y_cat = y

In [ ]:
hitsory = model.fit(x=x,
    y=y_cat, 
    batch_size=512,
    epochs=100,
    verbose=1, 
    callbacks=callbacks,
    validation_split=0.1, 
#     validation_data=[xval, yval], 
    shuffle=True, 
#     class_weight=class_weight, 
    sample_weight=None, 
    initial_epoch=0)

In [ ]:
w = np.where(y==0)[0]

In [ ]:
len(w)

In [ ]:
a = model.evaluate(x[w], y[w])

In [ ]:
a

In [ ]:

len(np.where(a>0.5)[0])

In [ ]:
len(x)

In [ ]:
test_session_path = "/home/wc-gpu/MasterThesis/session_data/122923_testing"

In [ ]:
model.save()